In [1]:
from pyspark.sql import SparkSession, types

spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

24/05/22 12:13:43 WARN Utils: Your hostname, DESKTOP-RJFPN4L resolves to a loopback address: 127.0.1.1; using 172.24.64.142 instead (on interface eth0)
24/05/22 12:13:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/22 12:13:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/22 12:13:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [46]:
green = spark.read.parquet('../data/pq/green/*/*/')
green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-29 13:42:10|  2020-01-29 13:48:24|                 N|         1|           7|         193|              3|         0.91|        6.0|  0.0|    0.

In [35]:
yellow = spark.read.parquet('../data/pq/yellow/*/*/')
yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       2| 2020-01-08 13:40:30|  2020-01-08 13:55:37|              1|         2.61|         1|                 N|         234|         237|           1|       12.0|  0.0|    0.5|      3.06|         0.0|                  0.3

In [42]:
yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [40]:
green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [50]:
green = green \
        .withColumnRenamed('lpep_pickup_datetime','pickup_datetime') \
        .withColumnRenamed('lpep_dropoff_datetime','dropoff_datetime') 
green.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [52]:
yellow = yellow \
        .withColumnRenamed('tpep_pickup_datetime','pickup_datetime') \
        .withColumnRenamed('tpep_dropoff_datetime','dropoff_datetime') 
yellow.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [54]:
same_cols = set(green.columns) & set(yellow.columns)

In [55]:
col_list = []
for col in green.columns:
    if col in same_cols:
        col_list.append(col)

In [56]:
col_list

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

In [60]:
from pyspark.sql import functions as F

In [68]:
green = green \
        .select(col_list) \
        .withColumn('service_type',F.lit('green'))

In [69]:
yellow = yellow \
        .select(col_list) \
        .withColumn('service_type',F.lit('yellow'))

In [70]:
trips_data = green.unionAll(yellow)

In [71]:
trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [73]:
trips_data.createOrReplaceTempView('trips_data')

In [78]:
spark.sql("""
SELECT service_type,count(1) 
FROM trips_data
group by service_type

""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [92]:
green.take(1)

[Row(VendorID=2, pickup_datetime=datetime.datetime(2020, 1, 29, 13, 42, 10), dropoff_datetime=datetime.datetime(2020, 1, 29, 13, 48, 24), store_and_fwd_flag='N', RatecodeID=1, PULocationID=7, DOLocationID=193, passenger_count=3, trip_distance=0.91, fare_amount=6.0, extra=0.0, mta_tax=0.5, tip_amount=1.36, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=8.16, payment_type=1, congestion_surcharge=0.0, service_type='green')]

In [84]:
rdd = green \
    .select('pickup_datetime','PULocationID','total_amount') \
    .rdd

In [106]:
rows = rdd.take(10)
row = rows[0]

In [107]:
row

Row(pickup_datetime=datetime.datetime(2020, 1, 29, 13, 42, 10), PULocationID=7, total_amount=8.16)

In [113]:
from datetime import datetime
start = datetime(year=2020,month=1,day=1)

def start_date(row):
    return row.pickup_datetime>=start

def prepare_for_grouping(row):
    hour = row.pickup_datetime.replace(minute=0,second=0,microsecond=0)
    zone = row.PULocationID
    key = (hour,zone)
    
    amount = row.total_amount
    count = 1
    value = (amount,count)
    
    return (key,value)

def calculate_revenue(left_value,right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value

    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    output_value = (output_amount, output_count)
    return output_value
    
from collections import namedtuple
RevenueRow = namedtuple('RevenueRow',['hour','zone','revenue','count'])

def unwrap(row):
    return RevenueRow(hour=row[0][0],zone= row[0][1],revenue= row[1][0],count= row[1][1])

rdd\
    .filter(start_date)\
    .map(prepare_for_grouping)\
    .reduceByKey(calculate_revenue)\
    .map(unwrap)\
    .toDF() \
    .show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-27 09:00:00| 207|               3.3|    1|
|2020-01-07 16:00:00|  97|            625.44|   34|
|2020-01-29 22:00:00| 129|244.43000000000006|   19|
|2020-01-21 15:00:00|  41| 512.7600000000002|   39|
|2020-01-27 22:00:00|  95|232.40000000000003|   20|
|2020-01-11 06:00:00| 179|             42.35|    2|
|2020-01-19 20:00:00|  97|            225.69|   19|
|2020-01-29 21:00:00| 250|             72.43|    2|
|2020-01-19 13:00:00|  47|             50.96|    2|
|2020-01-31 16:00:00|  74|1166.3499999999992|   71|
|2020-01-08 14:00:00| 212|             54.91|    4|
|2020-01-31 19:00:00|  25|415.56000000000006|   26|
|2020-01-25 05:00:00|  82|            147.67|   10|
|2020-01-11 22:00:00| 260|228.85000000000005|   20|
|2020-01-08 09:00:00| 179|156.99999999999997|    8|
|2020-01-17 16:00:00|  75|1793.0599999999981|  109|
|2020-01-06 